In [1]:
import ast
from pathlib import Path
from mutation import maker
from mutation import controller
from mutation import transformers

In [2]:
pkg_dir = Path("firstmodule")

In [3]:
src_files = controller.get_py_files(pkg_dir)
print(*src_files, sep="\n")

/home/zassoc/Github/m/firstmodule/__init__.py
/home/zassoc/Github/m/firstmodule/run.py


In [4]:
trees, targets = controller.build_src_trees_and_targets(pkg_dir)

2018-12-26 08:36:53,258 - mutation.controller - INFO - Creating AST from: /home/zassoc/Github/m/firstmodule/__init__.py
2018-12-26 08:36:53,260 - mutation.controller - INFO - Get mutation targets from AST.
2018-12-26 08:36:53,261 - mutation.controller - INFO - Creating AST from: /home/zassoc/Github/m/firstmodule/run.py
2018-12-26 08:36:53,261 - mutation.controller - INFO - Get mutation targets from AST.


In [5]:
sample_space = controller.get_sample_space(targets)

In [6]:
print(*sample_space, sep="\n")

('/home/zassoc/Github/m/firstmodule/run.py', LocIndex(ast_class='BinOp', lineno=10, col_offset=11, op_type=<class '_ast.Add'>))
('/home/zassoc/Github/m/firstmodule/run.py', LocIndex(ast_class='BinOp', lineno=6, col_offset=11, op_type=<class '_ast.Add'>))


In [15]:
for src, idx in sample_space:
    print(idx.op_type)
    mutants = transformers.get_mutations_for_target(idx)
    print(mutants)

<class '_ast.Add'>
{<class '_ast.Pow'>, <class '_ast.Mult'>, <class '_ast.Sub'>, <class '_ast.Div'>}
<class '_ast.Add'>
{<class '_ast.Pow'>, <class '_ast.Mult'>, <class '_ast.Sub'>, <class '_ast.Div'>}


In [10]:
l.ast_class

'BinOp'

In [11]:
BINOP_TYPES = {
    ast.Add, ast.Sub,
    ast.Div, ast.Mult, ast.Pow,
}

OTHER_TYPES = {
    ast.Eq, ast.NotEq, ast.Lt, ast.LtE
}

In [14]:
op_sets = [BINOP_TYPES, OTHER_TYPES]

In [15]:
for opset in op_sets:
    if l.op_type in opset:
        mutation_ops = opset.copy()
        mutation_ops.remove(l.op_type)

In [16]:
mutation_ops

{_ast.Div, _ast.Mult, _ast.Pow, _ast.Sub}

In [8]:
pprint(targets)

{'/home/zassoc/Github/m/firstmodule/run.py': [LocIndex(ast_class='BinOp', lineno=10, col_offset=11, op_type=<class '_ast.Add'>),
                                              LocIndex(ast_class='BinOp', lineno=6, col_offset=11, op_type=<class '_ast.Add'>)]}


In [9]:
sample_space = []

for k, v in targets.items():
    for tgt in v:
        sample_space.append((k, tgt))

In [10]:
sample_space

[('/home/zassoc/Github/m/firstmodule/run.py',
  LocIndex(ast_class='BinOp', lineno=10, col_offset=11, op_type=<class '_ast.Add'>)),
 ('/home/zassoc/Github/m/firstmodule/run.py',
  LocIndex(ast_class='BinOp', lineno=6, col_offset=11, op_type=<class '_ast.Add'>))]

In [40]:
src_files[-1].stem.startswith("test_")

False

In [11]:
tree = transformers.get_ast_from_src(src_files[-1])

In [12]:
tree.body

In [13]:
targets = maker.get_mutation_targets(tree)

In [14]:
targets

{LocIndex(ast_class='BinOp', lineno=6, col_offset=11, op_type=<class '_ast.Add'>),
 LocIndex(ast_class='BinOp', lineno=10, col_offset=11, op_type=<class '_ast.Add'>)}

In [16]:
t1 = targets.pop()

In [17]:
t1

LocIndex(ast_class='BinOp', lineno=6, col_offset=11, op_type=<class '_ast.Add'>)

In [19]:
targets

{LocIndex(ast_class='BinOp', lineno=10, col_offset=11, op_type=<class '_ast.Add'>)}

In [20]:
t1.op_type

_ast.Add

In [33]:
# Add | Sub | Mult | MatMult | Div | Mod | Pow | LShift

# | RShift | BitOr | BitXor | BitAnd | FloorDiv

In [22]:
binop_types = {
    ast.Add,
    ast.Sub,
    ast.Div,
    ast.Mult,
    ast.Pow,
}

In [36]:
while targets:
    print(targets.pop())

LocIndex(ast_class='BinOp', lineno=10, col_offset=11, op_type=<class '_ast.Add'>)


In [23]:
t1.op_type in binop_types

True

In [29]:
tops = binop_types.copy()

In [31]:
tops.remove(t1.op_type)

In [32]:
tops

{_ast.Div, _ast.Mult, _ast.Sub}

In [ ]:
class RewriteAddSub(ast.NodeTransformer):
    
    def __init__(self, readonly):
        self.readonly = readonly
        self.locs = []
        super()
        
    def visit_BinOp(self, node):
        self.generic_visit(node)
        
        self.locs.append(node.lineno)
        if self.readonly:
            print("Read only", node.lineno, node.col_offset, node.op)    
            self.locs.append(node.op)
            return node
        
        else:
            print("Transform to sub")
            return ast.copy_location(
                ast.BinOp(left=node.left,
                          op=ast.Sub(),
                          right=node.right),
                node)

In [ ]:
ronly = RewriteAddSub(readonly=True)

In [ ]:
t2 = ronly.visit(tree)

In [ ]:
ronly.locs

In [ ]:
type(ronly.locs[1])

In [ ]:
isinstance(ronly.locs[1], type(ronly.locs[1]))

In [ ]:
repr(type(ronly.locs[1]))